In [ ]:
import load_data 
import model_utils
import run_network
import network 
import matplotlib.pyplot as plt
import numpy as np
import torch
import cv2 as cv
import os
from torch import nn, optim
from torchvision import transforms, utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from skimage.measure import compare_ssim as ssim
import csv
%matplotlib inline

In [ ]:
path = './data/'
filename = 'dataset_batch_3.mat'
train_input_dataset,train_output_dataset = load_data.load_dataset(path,filename)
print(train_input_dataset.shape,train_output_dataset.shape)

In [ ]:
path = './data/'
filename = 'dataset_test.mat'
test_input_dataset,test_output_dataset = load_data.load_dataset(path,filename)
print(test_input_dataset.shape,test_output_dataset.shape)

In [ ]:
inputs_nums = [25,50,75,100,125,150,175,200]
batch_size = 40
device = torch.device("cuda:0")

In [ ]:
training_dataset = load_data.DeconvolutionDataset(train_input_dataset,train_output_dataset,
                                                  transform=transforms.Compose([load_data.ImageSelect(inputs_nums[7]),
                                                                              load_data.ToTensor()]))
training_dataloader = DataLoader(training_dataset, batch_size=batch_size,shuffle=True, num_workers=6)

In [ ]:
model = network.TestNet()
model.to(device)
criterion = nn.SmoothL1Loss()

optimizer = optim.Adam(model.parameters(), lr=1e-6)
num_epochs = 50000

run_network.train(training_dataloader,model,criterion, optimizer, num_epochs,device)

model_utils.save_model(model, optimizer, path = "./model/", filename = 'inputs_200_dataset_1189_epch_50000_testnet.pth')
    
test_data = np.asarray(np.transpose(test_input_dataset[:4,:inputs_nums[7],:,:],(0,1,2,3)),dtype=np.float32)
test_data = torch.from_numpy(test_data)
    
predict_test = run_network.test(test_data,model,device)

In [ ]:
model_utils.save_model(model, optimizer, path = "./model/", filename = 'inputs_200_dataset_1189_epch_6600_testnet.pth')
test_data = np.asarray(np.transpose(test_input_dataset[:4,:inputs_nums[7],:,:],(0,1,2,3)),dtype=np.float32)
test_data = torch.from_numpy(test_data)
    
predict_test = run_network.test(test_data,model,device)

In [ ]:
print(predict_test.shape)

In [ ]:
plt.imshow(predict_test[3,0])

In [ ]:
plt.imshow(test_output_dataset[3,:,:])